In [ ]:
## running in microsoft's planetary computer
import pystac_client
import planetary_computer
import rioxarray as rio
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from shapely import Point, box
import osmnx as ox
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from matplotlib_scalebar import scalebar
from xrspatial import hillshade
plt.rcParams['font.family'] = 'DejaVu Sans Mono'

In [ ]:
# set projection
# define aoi as lat, lon point. reproject to polar stereo then buffer
prj = ccrs.SouthPolarStereo()
aoi = gpd.GeoSeries(Point(161.571,-77.432), crs=4326)
aoi_prj = aoi.to_crs(prj)
aoi_buff_prj = aoi_prj.buffer(60_000)
aoi_small_buff_prj = aoi_prj.buffer(30_000)
aoi_buff = aoi_buff_prj.to_crs(4326) # buffer back in lat long for pystac search
west, south, east, north = aoi_buff.total_bounds

In [ ]:
## get contours in aoi from copernicus global DEM hosted on planetary computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace)

search = catalog.search(collections=['cop-dem-glo-30'],
                        intersects=aoi_buff[0])
items = list(search.get_items())

# if aoi crosses multiple DEM tiles stitch 'em together
if len(items) > 1:
    data = []
    for item in items:
        signed_asset = planetary_computer.sign(item.assets["data"])
        with rio.open_rasterio(signed_asset.href) as f:
            data.append(f.squeeze().drop_vars("band").rename('z'))
    dem = ((xr.merge(data))
           .rio.set_crs(4326)
           .rio.write_transform()
           .rio.clip_box(*aoi_buff.total_bounds))['z']
    f.close()
else:
    signed_asset = planetary_computer.sign(items[0].assets["data"])
    dem =  (rio.open_rasterio(signed_asset.href)
            .squeeze()
            .rio.set_crs(4326)
            .rio.write_transform()
            .rio.clip_box(*aoi_buff.total_bounds)).rename('z')

In [ ]:
## create envelope around small circle
frame = box(*aoi_small_buff_prj.buffer(2000)[0].bounds)

# reproject and clip to box
dem_prj = dem.rio.reproject(prj, resolution=30, nodata=np.nan).rio.clip_box(*frame.bounds)

# dump out arrays (for contouring)
Z = dem_prj.values
X,Y = np.meshgrid(dem_prj.x.values, dem_prj.y.values)

# hill shade and clipped to circle hillshade
hs = hillshade(dem_prj)
hs_clip = hs.rio.clip(aoi_small_buff_prj)

In [ ]:
## get features OSM
features = ox.features_from_bbox(north=north, south=south,
                                 east=east, west=west,
                              tags={'natural':['peak','water','bare_rock',
                                               'cliff','scree']})

# reproject and clip to circle
features = features.to_crs(prj).clip(aoi_small_buff_prj).reset_index()

# separate out each feaure tag
bare_rock = features.loc[features['natural']=='bare_rock'].copy()
water = features.loc[features['natural']=='water'].copy()
cliff = features.loc[features['natural']=='cliff'].copy()
scree = features.loc[features['natural']=='scree'].copy()
peaks = features.loc[features['natural']=='peak'].copy()

In [ ]:
### plotting
fig, ax = plt.subplots(figsize=[12,12],
                       subplot_kw={'projection':prj})

bare_rock.plot(ax=ax, fc='wheat', ec='none')

## contours every 100 m, with labels every 500 m
ax.contour(X,Y,Z,
           levels=[i for i in range(0,3000,100)], #  if i%500 !=0
           colors='k',
           linewidths = 0.1)

cplot = ax.contour(X,Y,Z,
                   levels = range(0,3000,500),
                   linewidths = 0.3,
                   colors = 'k')
ax.clabel(cplot, cplot.levels, fontsize=6,zorder=2)

## faint hillshade everywhere
hs.plot(ax=ax,
        cmap='Greys',
        alpha=0.05,
        add_colorbar=False)
# slightly less faint hillshade within circle
hs_clip.plot(ax=ax,
             cmap='Greys',
             alpha=0.3,
             add_colorbar=False)

## vector features
water.plot(ax=ax,
           fc='tab:blue',
           ec='none')
scree.plot(ax=ax,
           hatch='......', 
           fc='lightgray',
           ec='gray',
           alpha=0.7,
           linewidth=0)
cliff.plot(ax=ax,
           color='k',
           linewidth=0.2)
cliff.get_coordinates().plot.scatter(x='x',
                                     y='y',
                                     ax=ax,
                                     marker='3',
                                     c='gray',
                                     s=2)
peaks.plot(ax=ax,
           marker='^',
           markersize=20,
           fc='none',
           ec='k')

ax.set_axis_off()

## grid line nonsense
gl = ax.gridlines(crs=ccrs.PlateCarree(),
                  draw_labels=True,
                  color='grey',
                  linewidth=0.8,
                  zorder=1)
gl.xlocator = mticker.FixedLocator([160,161,162,163])
gl.ylocator = mticker.FixedLocator([-77.5,-77.25])
gl.rotate_labels = False
gl.y_inline=False
gl.bottom_labels=True
gl.right_labels=True
gl.top_labels=False
gl.left_labels=False
gl.xpadding = -5
gl.ypadding = -5
gl.xlabel_style = {'ha':'left'}
gl.ylabel_style = {'rotation':20}

ax.add_artist(scalebar.ScaleBar(1, location='upper right'))

ax.set_title('McMurdo Dry Valleys\n--:--\nAntarctica',
             x=0.12, y=0.92,
             ha='center',
             bbox={'facecolor':'w', 'edgecolor':'none','alpha':0.8},
             zorder=3)

ax.annotate(text='Data: Copernicus Global Digital Elevation Model (ESA) & OpenStreetMap',
            xy=(0.99,-0.02), 
            xycoords='axes fraction', 
            ha='right',
            c='k', 
            fontsize=10,
            font='DejaVu Sans Mono')

ax.annotate(text='by:tlohde',
            xy=(0.01,-0.02), 
            xycoords='axes fraction',
            ha='left',
            fontsize=10,
            font='DejaVu Sans Mono')

fig.savefig('day25.png', bbox_inches='tight', dpi=300)